<a href="https://colab.research.google.com/github/itsmepraks/SEAS_Search/blob/main/nb/Llama3.1_(8B)-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup Environment

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http:/

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Preparation
We load the GWU Computer Science course data from `data/combined_courses.csv` and format it into instruction-response pairs for fine-tuning.


In [4]:
%pip install pandas

In [5]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# 1. Load the JSONL dataset
dataset = load_dataset("json", data_files="course_finetune.jsonl", split="train")

# 2. Setup ChatML template (Standard for Llama 3)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, etc.
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"}, # ShareGPT style mapping
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# 3. Format the dataset
dataset = dataset.map(formatting_prompts_func, batched = True)

print(dataset[0]["text"])

Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


Map:   0%|          | 0/1992 [00:00<?, ? examples/s]

<|im_start|>system
You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026.<|im_end|>
<|im_start|>user
Tell me about CSCI 1012.<|im_end|>
<|im_start|>assistant
The course CSCI 1012: Introduction to Programming with Python is taught by Goldfrank, J. It meets on M 03:45PM - 05:00PM in 1957 E 213. The status is OPEN (CRN: 44900).<|im_end|>



### Model Training
We configure the `SFTTrainer` and begin fine-tuning the model on our custom dataset.

In [6]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1992 [00:00<?, ? examples/s]

In [7]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.135 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,992 | Num Epochs = 3 | Total steps = 747
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,3.777900
2,3.817500
3,3.682700
4,3.503300
5,2.782800
6,2.524800
7,2.238400
8,1.722600
9,1.506100
10,1.173900


Unsloth: Will smartly offload gradients to save VRAM!


In [9]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1233.1831 seconds used for training.
20.55 minutes used for training.
Peak reserved memory = 7.135 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 18.037 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Inference
We test the model with queries about GWU CS courses to verify it has learned the new data.

In [13]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "system", "value": "You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026."},
    {"from": "human", "value": "Who Teaches Machine Learning?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
import torch
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, attention_mask = torch.ones_like(inputs))

<|im_start|>system
You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026.<|im_end|>
<|im_start|>user
Who Teaches Machine Learning?<|im_end|>
<|im_start|>assistant
Machine Learning (CSCI 6364) is taught by Feng, S.<|im_end|>


### Save Model
We save the fine-tuned LoRA adapters locally.

In [ ]:
# 1. Save LoRA adapters locally (Required for other steps)
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("LoRA adapters saved to 'lora_model'.")

#### Test Saved Model
Reload the saved model to confirm it works.

In [15]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "system", "value": "You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026."},
    {"from": "human", "value": "What are the courses that I can take if I have not taken introduction courses?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
import torch
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, attention_mask = torch.ones_like(inputs))

<|im_start|>system
You are a helpful assistant providing information about GWU Computer Science courses for Spring 2026.<|im_end|>
<|im_start|>user
What are the courses that I can take if I have not taken introduction courses?<|im_end|>
<|im_start|>assistant
You can take Research.<|im_end|>


### Export Options (Float16 / GGUF)
Options to export the model for different serving backends (vLLM, llama.cpp).

In [ ]:
# 2. Export to Hugging Face (Merged Model)
# Options: "merged_16bit" (High precision), "merged_4bit" (Smaller), "lora" (Adapters only)
save_method = "merged_16bit"
push_to_hub = False
hf_repo_name = "username/my-fine-tuned-model"
hf_token = "hf_..." # Get token from https://huggingface.co/settings/tokens

if push_to_hub:
    print(f"Pushing {save_method} to Hub...")
    model.push_to_hub_merged(hf_repo_name, tokenizer, save_method=save_method, token=hf_token)
else:
    print(f"Saving {save_method} locally...")
    model.save_pretrained_merged("merged_model", tokenizer, save_method=save_method)

In [ ]:
# 3. Export to GGUF (for Ollama, LM Studio)
# Common options: "q4_k_m" (Balanced), "q8_0" (High quality), "f16" (Uncompressed)
quantization_method = "q4_k_m"
push_to_hub = False
hf_repo_name = "username/my-gguf-model"
hf_token = "hf_..."

if push_to_hub:
    print(f"Pushing {quantization_method} GGUF to Hub...")
    model.push_to_hub_gguf(hf_repo_name, tokenizer, quantization_method=quantization_method, token=hf_token)
else:
    print(f"Saving {quantization_method} GGUF locally...")
    model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method=quantization_method)